In [5]:
import numpy as np # linear algebra
import pandas as pd
# from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
# tqdm.pandas()

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, TimeDistributed, LeakyReLU, Conv1D, BatchNormalization, MaxPooling1D, AveragePooling1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras import layers, optimizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

import pickle

%matplotlib inline
import matplotlib.pyplot as plt

In [6]:
cluster=0
with open(f'D:\\庫存健診開發\\data\\Training\\processed\\Cluster_{cluster}_classification_minmax0_Weekly', 'rb') as fp:
    load_list = pickle.load(fp)
X_train, Y_train, X_val, Y_val, X_test, Y_test = load_list[0], load_list[1], load_list[2], load_list[3], load_list[4], load_list[5]
def get_dependent_variable(Y_train, Y_val, Y_test):
        training = [item[2] for item in Y_train]
        val = [item[2] for item in Y_val]
        testing = [item[2] for item in Y_test]

        return training, val, testing


def categorical_transform(data):

    onehot_encoder = OneHotEncoder(sparse=False)

    d = np.array(data).reshape(len(data), 1)
    Y = onehot_encoder.fit_transform(d)

    return Y

y_train, y_val, y_test = get_dependent_variable(Y_train, Y_val, Y_test)
y_train_encode, y_val_encode, y_test_encode = categorical_transform(y_train), categorical_transform(y_val), categorical_transform(y_test)

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and

In [19]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(np.array(X_train).shape[1], np.array(X_train).shape[2]), padding='causal'))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='causal'))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='causal'))
model.add(Flatten(name='feature'))
model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',
                optimizer=optimizers.Nadam(lr=0.001),
                metrics=['accuracy'])
    
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
model.fit(np.array(X_train), y_train_encode, validation_data=(np.array(X_val), y_val_encode), batch_size = 256, epochs=30, verbose=2)

Train on 197067 samples, validate on 27592 samples
Epoch 1/30
197067/197067 - 5s - loss: 0.6747 - acc: 0.5838 - val_loss: 0.6717 - val_acc: 0.5903
Epoch 2/30
197067/197067 - 4s - loss: 0.6666 - acc: 0.5990 - val_loss: 0.6703 - val_acc: 0.5878
Epoch 3/30
197067/197067 - 4s - loss: 0.6645 - acc: 0.6010 - val_loss: 0.6688 - val_acc: 0.5897
Epoch 4/30
197067/197067 - 4s - loss: 0.6633 - acc: 0.6027 - val_loss: 0.6724 - val_acc: 0.5827
Epoch 5/30
197067/197067 - 4s - loss: 0.6627 - acc: 0.6039 - val_loss: 0.6780 - val_acc: 0.5850
Epoch 6/30
197067/197067 - 4s - loss: 0.6620 - acc: 0.6043 - val_loss: 0.6708 - val_acc: 0.5885
Epoch 7/30
197067/197067 - 4s - loss: 0.6612 - acc: 0.6058 - val_loss: 0.6715 - val_acc: 0.5921
Epoch 8/30
197067/197067 - 4s - loss: 0.6610 - acc: 0.6063 - val_loss: 0.6698 - val_acc: 0.5881
Epoch 9/30
197067/197067 - 4s - loss: 0.6606 - acc: 0.6057 - val_loss: 0.6695 - val_acc: 0.5918
Epoch 10/30
197067/197067 - 4s - loss: 0.6597 - acc: 0.6078 - val_loss: 0.6683 - val_

In [14]:
intermediate_layer_model = Model(inputs=model.input, outputs=model.get_layer('feature').output)
training_feature = intermediate_layer_model.predict(np.array(X_train))
val_feature = intermediate_layer_model.predict(np.array(X_val))
testing_feature = intermediate_layer_model.predict(np.array(X_test))

In [9]:
rf = RandomForestClassifier(n_estimators=100, 
                                                        max_depth=10,
                                                        max_features='auto',
                                                        min_samples_leaf=0.1, 
                                                        min_samples_split=0.1,
                                                        verbose=0, 
                                                        n_jobs=-1)

In [15]:
rf.fit(training_feature, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=0.1, min_samples_split=0.1,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [18]:
rf.score(testing_feature, y_test)

0.6211760873106369

In [35]:
Optimization.load_optimal_model()

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and

In [35]:
display(Optimization.results['CNN_params'].iloc[0])

{'Conv_layer': 2,
 'Dense_layer': 1,
 'Dense': [256],
 'dropout_ratio': [0.15, 0.25, 0],
 'filter': [100, 256],
 'kernel': [3, 3],
 'padding': ['causal', 'causal'],
 'learning_rate': 0.003,
 'batch': 256,
 'epochs': 1}

In [30]:
Optimization.load_optimal_param()

In [32]:
Optimization.results

,auc,acc,balance_acc,balance_auc,CNN_params,Model_params
0,0.552424,0.553566,0.849315,0.90411,"{'Conv_layer': 2, 'Dense_layer': 1, 'Dense': [...","{'classifier': 'RandomForest', 'max_depth': 5,..."
